# **Sentiment Analysis**

We'll start by loading in the IMDB movie review dataset from keras.

This dataset contains 25,000 reviews from IMDB where each one is already preprocessed and has a label as either positive or negative. Each review is encoded by integers that represents how common a word is in the entire dataset. For example, a word encoded by the integer 3 means that it is the 3rd most common word in the dataset.

### **Load Data**

In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

17473536/17464789 [==============================] - 3s 0us/step


### **Preprocessing**

We must feed our RNN with the same length of data. Hence we will have to either cut our sentences to a maximum of 250 words or add in the necessary zeros to make it 250 words.

In [2]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

### **Create Model**

In [31]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [32]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 32)          2834688   
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


### **Training**

In [29]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 34s 53ms/step - loss: 7.1266 - acc: 0.5031 - val_loss: 7.7192 - val_acc: 0.4938
Epoch 2/10
625/625 [==============================] - 30s 48ms/step - loss: 7.6010 - acc: 0.5016 - val_loss: 7.7192 - val_acc: 0.4938
Epoch 3/10
625/625 [==============================] - 34s 54ms/step - loss: 7.6010 - acc: 0.5016 - val_loss: 7.7192 - val_acc: 0.4938
Epoch 4/10
625/625 [==============================] - 34s 54ms/step - loss: 7.6010 - acc: 0.5016 - val_loss: 7.7192 - val_acc: 0.4938
Epoch 5/10
625/625 [==============================] - 36s 57ms/step - loss: 7.6010 - acc: 0.5016 - val_loss: 7.7192 - val_acc: 0.4938
Epoch 6/10
625/625 [==============================] - 36s 58ms/step - loss: 7.6010 - acc: 0.5016 - val_loss: 7.7192 - val_acc: 0.4938
Epoch 7/10
625/625 [==============================] - 45s 72ms/step - loss: 7.6010 - acc: 0.5016 - val_loss: 7.7192 - val_acc: 0.4938
Epoch 8/10
625/625 [==============================] - 43s 69ms

In [30]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 11s 14ms/step - loss: 7.6246 - acc: 0.5000
[7.624584197998047, 0.5]


### **Making Predictions**

Convert any of our text to encoded text

In [7]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

1654784/1641221 [==============================] - 1s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [8]:
# while were at it lets make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

that movie was just amazing so amazing


In [10]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

[0.91663325]
[0.43514603]


In [17]:
neutral_review = "The movie was alright, may not be the best but it is not the worst!"
predict(neutral_review)

[0.6718383]


In [18]:
positive_review_2 = "The movie was inspirational and very heartwarming. I love the colors and the details they added in the movie!"
predict(positive_review_2)

[0.8534243]


In [26]:
negative_review_2 = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review_2)

[0.43514603]
